In [30]:
import pandas as pd

user_data = pd.read_csv("all_user_data_cc.csv")

In [48]:
user_data.drop(user_data.columns[[0]], axis=1, inplace=True)
user_data.head()

,Unnamed: 0.1,id,name,editcount,status
0,0,15,!!,1540,0
1,1,228,!dea4u,6722,0
2,2,387,"""Country"" Bushrod Washington",1652,0
3,3,582,"""Yahia""",1023,0
4,4,829,$1LENCE D00600D,18642,0


In [49]:
user_data = user_data[user_data['status']>0]
user_data = user_data[:1500]  # same as df.head(10)

In [50]:
user_data.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,Unnamed: 0.1,id,editcount,status
count,0,0,0,0
mean,nan,nan,nan,nan
std,nan,nan,nan,nan
min,nan,nan,nan,nan
25%,nan,nan,nan,nan
50%,nan,nan,nan,nan
75%,nan,nan,nan,nan
max,nan,nan,nan,nan


In [5]:
user_data_c.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

NameError: name 'user_data_c' is not defined

In [23]:
user_data.to_csv("all_user_data_cc.csv")

In [ ]:
new_column = ['status']

user_data_c = user_data_c.reindex(columns=user_data_c.columns.tolist() + new_column)


user_data_c.drop(user_data_c.columns[[0]], axis=1, inplace=True)
user_data_c.to_csv("../all_user_data_c.csv")

In [11]:
user_data.head()

,id,name,editcount,status
0,15,!!,1540,1.0
1,228,!dea4u,6722,1.0
2,387,"""Country"" Bushrod Washington",1652,1.0
3,582,"""Yahia""",1023,1.0
4,829,$1LENCE D00600D,18642,1.0


In [34]:

import os.path
import numpy as np
from trust_score import TrustScore
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import json

y_pred = []
y_test=[]
cou = 0

for row in user_data.iterrows():
    path = 'user_data/rev_list_' + str(row[1]['id']) + '-d.json'
    #print(path)
    try:
        with open(path, 'r') as infile:
            data = json.loads(infile.read())

            if len(data) > 0:
                for d in data:
                    del d['next_rev']

                graph_for = "longevityRev"

                series = pd.DataFrame(data=data)
                series = series[['pageid', 'timestamp', graph_for]]
                series = series[series.longevityRev >= 0]
                if len(series) > 9:
                    print('User ID: ', str(row[1]['id']))
                    series['longevityRevA'] = series.iloc[:,2].rolling(window=4).mean()
                    series.iloc[0:3,3]=series.iloc[0:3,2]
                    series['longevityRev'] = series['longevityRevA']
                    #print(series.head())
                    series['Trust'] = TrustScore([series[graph_for],48]).calculate()
                    series['longevityRevN'] = series['longevityRev'].shift(-1)
                    series['longevityRevP'] = series['longevityRev'].shift(1)
                    series['longevityRevPP'] = series['longevityRev'].shift(2)
                    series = series[series.longevityRevP >= 0]
                    series = series[series.longevityRevN >= 0]
                    series = series[series.longevityRev >= 0]
                    #series = series[series.longevityRevPP >= 0]

                    seriesTrain = series[0:-1]
                    seriesTest = series[len(series)-1:]
                    #print(series)
                    regressor = LinearRegression()
                    regressor.fit(seriesTrain[['Trust', 'longevityRev','longevityRevP']], seriesTrain['longevityRevN'])

                    # To retrieve the intercept:
                    #print(regressor.intercept_)
                    # For retrieving the slope:
                    #print(regressor.coef_)

                    pred = regressor.predict(seriesTest[['Trust', 'longevityRev','longevityRevP']])
                    print('Data Count: ', len(seriesTrain))
                    print('Test Data: ')
                    print(seriesTest[['Trust', 'longevityRev','longevityRevP']])
                    print('predicted: ',pred, ' actual: ', seriesTest['longevityRevN'].values[0])
                    print("##########################")
                    if(pred>=0):
                        y_test.append(seriesTest['longevityRevN'].values[0])
                        y_pred.append(pred)
    except Exception as e:
        #print('error :',e.message)
        cou= cou+1

print(str(cou))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('R2 Score:', metrics.r2_score(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

User ID:  15
Data Count:  169
Test Data: 
        Trust  longevityRev  longevityRevP
346  0.076948           3.0            3.0
predicted:  [7.00023359]  actual:  6.25
##########################
User ID:  1400
Data Count:  14
Test Data: 
       Trust  longevityRev  longevityRevP
96  0.783069          36.0           36.0
predicted:  [36.8626413]  actual:  48.0
##########################
User ID:  2346
Data Count:  58
Test Data: 
        Trust  longevityRev  longevityRevP
132  0.564606          27.0           27.0
predicted:  [27.69182686]  actual:  15.0
##########################
User ID:  3201
Data Count:  191
Test Data: 
        Trust  longevityRev  longevityRevP
308  0.598028          36.0          32.75
predicted:  [35.46870106]  actual:  36.0
##########################
User ID:  3763
Data Count:  40
Test Data: 
       Trust  longevityRev  longevityRevP
92  0.678094          25.5           30.0
predicted:  [27.94327845]  actual:  25.5
##########################
User ID:  3930
Data C

In [59]:
import pandas as pd

user_data = pd.read_csv("all_user_data_c_50_90.csv")

In [64]:
user_data.describe()
user_data.head()

,id,name,editcount,status
0,15,!!,1540,1
1,228,!dea4u,6722,1
2,387,"""Country"" Bushrod Washington",1652,0
3,582,"""Yahia""",1023,0
4,829,$1LENCE D00600D,18642,0


In [54]:
user_data['status'] = 0
user_data.drop(user_data.columns[[0]], axis=1, inplace=True)
user_data.head()

,id,name,editcount,status
0,15,!!,1540,0
1,228,!dea4u,6722,0
2,387,"""Country"" Bushrod Washington",1652,0
3,582,"""Yahia""",1023,0
4,829,$1LENCE D00600D,18642,0


In [63]:
user_data.drop(user_data.columns[[0]], axis=1, inplace=True)
user_data.to_csv("all_user_data_c_50_90.csv")